In [1]:
import CS_Indicators as CS
from brix import Indicator, Handler
import geopandas as gpd
import pandas as pd

In [2]:
table_name='providence_test'

### Load the Saved Data

In [3]:
geogrid=gpd.read_file('tables/{}/geogrid.geojson'.format(table_name))
zones=gpd.read_file('tables/{}/zones.geojson'.format(table_name))
zones['GEOID']=zones['GEOID'].astype(int)
zones=zones.set_index('GEOID')
simpop_df=pd.read_csv('tables/{}/simpop_df.csv'.format(table_name))

In [7]:
zones.columns

Index(['STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'NAMELSAD', 'MTFCC',
       'FUNCSTAT', 'area', 'AWATER', 'INTPTLAT', 'INTPTLON', 'x_centroid',
       'y_centroid', 'res_total', 'res_income_low', 'res_income_mid',
       'res_income_high', 'res_age_low', 'res_age_mid', 'res_age_high',
       'res_edu_no_highsch', 'res_edu_highsch', 'res_edu_some_college',
       'res_edu_bach_or_higher', 'emp_total', 'emp_income_low',
       'emp_income_mid', 'emp_income_high', 'emp_age_low', 'emp_age_mid',
       'emp_age_high', 'emp_edu_no_highsch', 'emp_edu_highsch',
       'emp_edu_some_college', 'emp_edu_bach_or_higher', 'emp_naics_11',
       'emp_naics_21', 'emp_naics_22', 'emp_naics_23', 'emp_naics_31-33',
       'emp_naics_42', 'emp_naics_44-45', 'emp_naics_48-49', 'emp_naics_51',
       'emp_naics_52', 'emp_naics_53', 'emp_naics_54', 'emp_naics_55',
       'emp_naics_56', 'emp_naics_61', 'emp_naics_62', 'emp_naics_71',
       'emp_naics_72', 'emp_naics_81', 'emp_naics_92', 'reference_are

In [4]:
zones.head()

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,NAMELSAD,MTFCC,FUNCSTAT,area,AWATER,INTPTLAT,...,emp_naics_62,emp_naics_71,emp_naics_72,emp_naics_81,emp_naics_92,reference_area,model_area,copy_GEOID,sim_area,geometry
GEOID,,,,,,,,,,,,,,,,,,,,,
440070031002,44,007,003100,2,Block Group 2,G5030,S,162044,0,+41.8403235,...,2.0,21.0,49.0,0.0,0.0,False,True,440070031002,False,"POLYGON ((-71.40922 41.83841, -71.40906 41.839..."
440070031003,44,007,003100,3,Block Group 3,G5030,S,141613,0,+41.8369783,...,2.0,15.0,5.0,2.0,0.0,False,True,440070031003,False,"POLYGON ((-71.40978 41.83535, -71.40973 41.835..."
440070031004,44,007,003100,4,Block Group 4,G5030,S,172471,0,+41.8325863,...,112.0,0.0,4.0,35.0,0.0,False,True,440070031004,False,"POLYGON ((-71.41133 41.83168, -71.41131 41.831..."
440070031005,44,007,003100,5,Block Group 5,G5030,S,508043,2607,+41.8382690,...,1065.0,16.0,172.0,236.0,2499.0,False,True,440070031005,False,"POLYGON ((-71.41745 41.83201, -71.41715 41.832..."
440070032001,44,007,003200,1,Block Group 1,G5030,S,200964,0,+41.8438499,...,0.0,0.0,13.0,17.0,0.0,False,True,440070032001,False,"POLYGON ((-71.39705 41.84132, -71.39697 41.841..."


In [5]:
simpop_df.head()

,age,earnings,industry,home_geoid,work_geoid
0,low,mid,trade_trans_util,440070001011,440070003001
1,mid,mid,other,440070001011,440070003002
2,high,low,other,440070001011,440070003005
3,mid,mid,other,440070001011,440070004001
4,mid,low,other,440070001011,440070007002


In [6]:
geogrid.head()

,id,interactive,name,x_centroid,y_centroid,geometry
0,0,Web,None,-71.429212,41.81517,"POLYGON ((-71.42951 41.81539, -71.42951 41.814..."
1,1,Web,None,-71.428610,41.81517,"POLYGON ((-71.42891 41.81539, -71.42891 41.814..."
2,2,Web,None,-71.428009,41.81517,"POLYGON ((-71.42831 41.81539, -71.42831 41.814..."
3,3,Web,None,-71.427407,41.81517,"POLYGON ((-71.42771 41.81539, -71.42771 41.814..."
4,4,Web,None,-71.426805,41.81517,"POLYGON ((-71.42711 41.81539, -71.42711 41.814..."


### Create and run the Cityscope model

In [5]:
H=Handler(table_name=table_name)
H.reset_geogrid_data()

In [9]:
d=CS.Density_Indicator(zones=zones)
p=CS.Proximity_Indicator(zones=zones, geogrid=geogrid)
m=CS.Mobility_indicator(zones, geogrid, table_name, simpop_df)
H.add_indicators([
    d, p, 
    m
])

# H.add_indicator(m)

Setting up Proximity Indicator
	 Getting central nodes
	 Finding all reachable geometries from each geometry
	 Calculating baseline scores
Dens Ind: 0:00:00.005466
Prox Ind: 0:00:00.105349
Prox HM: 0:00:00.005028


/Users/doorleyr/anaconda3/envs/brix/lib/python3.9/site-packages/brix/classes.py:575: UserWarning: Indicator Mobility already exists and will be overwritten
  warn(f'Indicator {indicatorName} already exists and will be overwritten')


Starting MM Update
Schedules and locations
Trip table
Route table


/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:672: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


DeckGL
Post ABM: <Response [200]>
Finished MM Update


In [ ]:
H.listen()

Starting MM Update
Schedules and locations
Trip table
Route table


/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:672: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


DeckGL
Post ABM: <Response [200]>
Finished MM Update
Dens Ind: 0:00:00.005202
Prox Ind: 0:00:00.107899
Prox HM: 0:00:00.005441
Starting MM Update
Schedules and locations
Trip table
Route table


/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:672: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


DeckGL
Post ABM: <Response [200]>
Finished MM Update
Dens Ind: 0:00:00.006239
Prox Ind: 0:00:00.109959
Prox HM: 0:00:00.004966
Starting MM Update
Schedules and locations
Trip table
Route table


/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:672: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


DeckGL
Post ABM: <Response [200]>
Finished MM Update
Dens Ind: 0:00:00.009228
Prox Ind: 0:00:00.336471
Prox HM: 0:00:00.010609
Starting MM Update
Schedules and locations
Trip table
Route table


/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/OpenCity/OpenCity.py:672: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


DeckGL
Post ABM: <Response [200]>
Finished MM Update
Dens Ind: 0:00:00.006785
Prox Ind: 0:00:00.261256
Prox HM: 0:00:00.009170
